# Curso de Machine Learning - Udemy

## Pré-processamento de Dados

### Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx
import yellowbrick

### Base de dados de crédito

* Fonte (adatpada): https://www.kaggle.com/laotse/credit-risk-dataset

In [ ]:
base_credit = pd.read_csv('Bases de dados/credit_data.csv')

- Importando a base de dados.

In [ ]:
base_credit

- Executando a base de dados.

In [ ]:
base_credit.head()

- Pegandos os primeiros registros da base de dados.

In [ ]:
base_credit.tail()

,clientid,income,age,loan,default
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0
1999,2000,69436.579552,56.152617,7378.833599,0


- Pegandos os últimos registros da base de dados.

In [ ]:
base_credit.describe()

- Retorna uma descrição em forma de contagem do elementos.

### Visualização dos dados

In [ ]:
np.unique(base_credit['default'], return_counts = True)


- `unique` retorna os valores únicos de uma coluna;
- `return_counts = True` retorna a quantidade dos valores que se encaixam no unique.

In [ ]:
sns.countplot(x = base_credit['default']);

- Gera um gráfico de barra para os valores pedidos;
- O ";" limpa a saída de textos poluídos.

In [ ]:
plt.hist(x =  base_credit['age']);

- Gera um histograma dos valores pedidos, nesse exemplo, um histograma com as idades na base de cŕedito.

In [ ]:
grafico = px.scatter_matrix(base_credit, dimensions = ['age', 'income', 'loan'], color = 'default')
grafico.show()

- Gera um gráfico interativo de acordo com os parâmetros inseridos

### Tratamento de valores inconsistentes

In [ ]:
base_credit.loc[base_credit['age'] < 0]

- Olhando as idades inválidas (abaixo de zero)

In [ ]:
base_credit.mean()

- Pegando a média dos valores da base de cŕedito, porém dessa forma, todos os valores estão tendo suas médias inclusas e só queremos as médias de idades para preencher os valores inválidos.

In [ ]:
base_credit['age'][base_credit['age'] > 0].mean()

- Forma correta de pegar a média sem considerar as idades inválidas

- Forma errada de alterar os dados inválidos pela média

   **base_credit.loc[base_credit['age'] < 0] = 40.92**

- Dessa forma, a linha inteira vai ser alterada para o valor inserido

In [ ]:
base_credit.loc[base_credit['age'] < 0, 'age'] = 40.92

- Forma correta de alterar dados inválidos

### Tratamentos de valores faltantes

In [ ]:
base_credit.isnull().sum()

- Forma de olhar os valores faltantes, se for 0 é válido, se for 1, então existe algum valor faltante

In [ ]:
base_credit.loc[pd.isnull(base_credit['age'])]

- Forma de localizar os valores nulos

In [ ]:
base_credit['age'].fillna(base_credit['age'].mean(), inplace = True)

- Forma de preencher os valores nulos com a média das idades

In [ ]:
base_credit.loc[(base_credit['clientid'].isin([29, 31, 32]))]

- Forma simples de localizar os elementos em um intervalo

### Divisão entre previsores e classe

No aprendizado de máquina supervisionado, os dados de entrada podem ser divididos em dois grupos
- X: são os atributos que vão ser utilizados para determinar a classe de saída. Esses atributos também podem ser chamados de previsores.
- Y: é o atributo para o qual se deseja fazer a predição do valor de saída (também chamado de atributo-alvo)

In [ ]:
X_credit = base_credit.iloc[:, 1:4].values

- Pegando os valores e as colunas especificadas nesse intervalo e colocando-os na variável `X`, nesse caso, `X_credit`.
- `values` é usado para converter os dados para um formato melhor de se trabalhar pelo computador.

In [ ]:
X_credit

- Mostrando os valores que estão atualmente na variável `X`.

In [ ]:
type(X_credit)

- Mostrando o tipo atual desse formato de dados.

In [ ]:
Y_credit = base_credit.iloc[:, 4].values

- Fazendo a atribuição da classe ou atributo-alvo em `Y`, nesse caso, `Y_credit`.

In [ ]:
Y_credit

- Mostrando os valores que estão atualmente na variável `Y`.

In [ ]:
type(Y_credit)

- Mostrando o tipo atual desse formato de dados.

### Escalonamento de valores

Os algoritmos de aprendizado de máquina tendem a favorecer os dados de tamanhos maiores e deixar de lado os dados com valores menores, atribuindo maior peso e importância para os dados com valores maiores.

Quase nunca isso é uma coisa boa, por isso, temos técnicas que ajudam a equalizar melhor os pesos que cada dado deve ter.

Os seguintes cálculos ajudam nesse problema: 

**Padronização** (Standardisation)

$x = \dfrac{x - média(x)}{desviopadrão(x)}$

**Normalização** (Normalization)

$x = \dfrac{x - mínimo(x)}{máximo(x) - mínimo(x)}$

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_credit = StandardScaler()
X_credit = scaler_credit.fit_transform(X_credit)

X_credit[:,0].min(), X_credit[:,1].min(), X_credit[:,2].min()

- A partir da biblioteca Sklearn, importamos a função que vai padronizar nossos dados
- Pegamos os menores valores de cada coluna (atributo) da nossa base de dados

In [ ]:
X_credit

- Podemos ver que agora dos dados da nossa base estão na mesma escala.
- Dizemos que esses dados estão escalonados

### Base de dados do censo

- Fonte: http://archive.ics.uci.edu/ml/datasets/adult

### Esploração dos dados 


In [ ]:
base_census = pd.read_csv('Bases de dados/census.csv')

In [ ]:
base_census

In [ ]:
base_census.describe()

Com a função `describe` podemos ver um quado geral dos dados da nossa base

In [ ]:
base_census.isnull().sum()

Usando `isnull` e `sum` podemos verificar se temos dados faltantes e/ou negativos, o que não é o caso para essa base de dados do census

### Visualização de dados

In [ ]:
np.unique(base_census['income'], return_counts = True)

- Usando a chamada acima, podemos pegar os valores referentes ao atributo `income (renda)` e a quantidade de dados que atendem a esses valores  

In [ ]:
sns.countplot(x = base_census['income']);

- Podemos considerar esses tipos de dados como `desbalanceados` por termos uma quantidade muito maior de um tipo de dado em relação ao outro

In [ ]:
plt.hist(x = base_census['age']);

In [ ]:
plt.hist(x = base_census['education-num']);

In [ ]:
plt.hist(x = base_census['hour-per-week']);

In [ ]:
grafico = px.treemap(base_census, path = ['workclass', 'age', 'income']);
grafico.show()

- Como mostrado acima, a função `treemap` mostra as hierarquias dos dados usando vários quadrados aninhados

In [ ]:
grafico = px.parallel_categories(base_census, dimensions = ['occupation', 'relationship']);
grafico.show()

- A função `parallel_categories` consegue relacionar os dados de dois atributos distintos e mostrar isso em um gráfico

### Divisão entre previsores e classes

In [ ]:
base_census.columns

In [ ]:
X_census = base_census.iloc[:, 0:14].values
X_census

- A função `iloc` é uma função do `pandas` usada para selecionar dados com base em sua posição numérica.
- `dataframe.iloc[linhas, colunas]`

- No caso, `X_census` está pegando todas as linhas e até a coluna 13, sem pegar a coluna 14 que é a `renda (income)`

In [ ]:
Y_census = base_census.iloc[:, 14].values
Y_census

### Tratamento de atributos categóricos

#### LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

- A função `fit_transform` ajusta o modelo de treinamento e em seguida aplica uma transformaçãono modelo
- A função `LabelEncoder` transforma `rótulos (dados categóricos)` em dados numéricos

In [ ]:
X_census[:, 1]

In [ ]:
label_encoder_teste = LabelEncoder()
label_encoder_teste

In [ ]:
teste = label_encoder_teste.fit_transform(X_census[:, 1])
teste

In [ ]:
X_census[0]

In [ ]:
label_encoder_workclass = LabelEncoder()
label_encoder_education = LabelEncoder()
label_encoder_marital = LabelEncoder()
label_encoder_occupation = LabelEncoder()
label_encoder_relationship = LabelEncoder()
label_encoder_race = LabelEncoder()
label_encoder_sex = LabelEncoder()
label_encoder_country = LabelEncoder()

In [ ]:
X_census[:, 1] = label_encoder_workclass.fit_transform(X_census[:, 1])
X_census[:, 3] = label_encoder_education.fit_transform(X_census[:, 3])
X_census[:, 5] = label_encoder_marital.fit_transform(X_census[:, 5])
X_census[:, 6] = label_encoder_occupation.fit_transform(X_census[:, 6])
X_census[:, 7] = label_encoder_relationship.fit_transform(X_census[:, 7])
X_census[:, 8] = label_encoder_race.fit_transform(X_census[:, 8])
X_census[:, 9] = label_encoder_sex.fit_transform(X_census[:, 9])
X_census[:, 13] = label_encoder_country.fit_transform(X_census[:, 13])

In [ ]:
X_census[0]

#### OneHotEncoder

In [ ]:
len(np.unique(base_census['workclass']))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
OneHotEncoder_census = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [1, 3, 5, 6, 7, 8, 9, 13])], remainder='passthrough')

In [ ]:
X_census = OneHotEncoder_census.fit_transform(X_census).toarray()

In [ ]:
X_census.shape

#### Escalonamento dos valores

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_census = StandardScaler()
X_census = scaler_census.fit_transform(X_census)

In [ ]:
X_census

### Divisão das bases em treinamento e teste

In [ ]:
from sklearn.model_selection import train_test_split

### Credit data

In [ ]:
X_credit_treinamento, X_credit_teste, Y_credit_treinamento, Y_credit_teste = train_test_split(X_credit, Y_credit, test_size=0.25, random_state=0)

### Census data

In [ ]:
X_census_treinamento, X_census_teste, Y_census_treinamento, Y_census_teste = train_test_split(X_census, Y_census, test_size=0.15, random_state=0)

### Salvar as variáveis

In [ ]:
import pickle

In [ ]:
with open('credit.pkl', mode='wb') as f:
    pickle.dump([X_credit_treinamento, Y_credit_treinamento, X_credit_teste, Y_credit_teste], f)

In [ ]:
with open('census.pkl', mode='wb') as f:
    pickle.dump([X_census_treinamento, Y_census_treinamento, X_census_teste, Y_census_teste], f) 

## Aprendizagem Bayesiana

### Naive Bayes (extra)

- O algoritmo ``Naive Bayes`` é um classificador probabilístico que assume que as ``características (features)`` são independentes entre si, daí o termo ``“naive” (ingênuo)``. Essa é uma simplificação feita para facilitar o cálculo das probabilidades condicionais necessárias para classificação.

$P(A|B) = \dfrac{P(B|A) \ P(A)}{P(B)}$

- Na prática, o algoritmo Naive Bayes é frequentemente utilizado para classificar textos, como na detecção de spam ou na categorização de documentos. Ele usa a frequência das palavras para calcular a probabilidade de um documento pertencer a uma determinada classe (por exemplo, spam ou não spam).

- Embora o algoritmo Naive Bayes seja simples e rápido, nem sempre é o classificador mais preciso, principalmente quando as características são altamente correlacionadas. No entanto, em muitos casos, o Naive Bayes pode fornecer resultados satisfatórios com um pequeno número de dados de treinamento.

In [ ]:
import pandas as pd

# Criando o dataframe de exemplo
dados = {
    'temperatura': [30, 25, 28, 18, 20, 22, 24, 28, 26, 30],
    'umidade': [85, 90, 78, 65, 75, 70, 80, 75, 80, 70],
    'jogar_tenis': ['Não', 'Não', 'Sim', 'Sim', 'Sim', 'Sim', 'Não', 'Sim', 'Sim', 'Não']
}

df = pd.DataFrame(dados)
print(df)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Separando as variáveis de entrada (temperatura e umidade) e o alvo (jogar_tenis)
X = df[['temperatura', 'umidade']]
y = df['jogar_tenis']

# Dividindo o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)

# Criando o modelo Naive Bayes Gaussiano
modelo = GaussianNB()

# Treinando o modelo
modelo.fit(X_train, y_train)

# Fazendo previsões
y_pred = modelo.predict(X_test)

# Calculando a acurácia do modelo
acuracia = accuracy_score(y_test, y_pred)
print(f'Acurácia: {acuracia * 100}%')

O ``test_size`` é um parâmetro que determina a proporção do conjunto de dados que será reservada para o conjunto de teste. Por exemplo, se test_size for definido como 0.2, isso significa que 20% dos dados serão usados como conjunto de teste, enquanto os 80% restantes serão usados como conjunto de treinamento. A escolha adequada do tamanho do conjunto de teste é importante para avaliar adequadamente o desempenho do modelo.

O ``random_state`` é um parâmetro opcional que permite que você fixe a semente (seed) usada pelo gerador de números aleatórios durante a divisão dos dados. Fixar a semente garante reprodutibilidade, ou seja, se você usar a mesma semente, obterá a mesma divisão de dados em diferentes execuções do código. Isso é útil para garantir resultados consistentes ao compartilhar código ou ao tentar depurar.

### Naive Bayes (exemplos do curso)

### Base risco de crédito

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
base_risco_credito = pd.read_csv('Bases de dados/risco_credito.csv')

In [ ]:
base_risco_credito

In [ ]:
X_risco_credito = base_risco_credito.iloc[:, 0:4].values
X_risco_credito

In [ ]:
Y_risco_credito = base_risco_credito.iloc[:, 4:5].values
Y_risco_credito

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder_historia = LabelEncoder()
label_encoder_divida = LabelEncoder()
label_encoder_garantias = LabelEncoder()
label_encoder_renda = LabelEncoder()

In [ ]:
X_risco_credito[:, 0] = label_encoder_historia.fit_transform(X_risco_credito[:, 0])
X_risco_credito[:, 1] = label_encoder_divida.fit_transform(X_risco_credito[:, 1])
X_risco_credito[:, 2] = label_encoder_garantias.fit_transform(X_risco_credito[:, 2])
X_risco_credito[:, 3] = label_encoder_renda.fit_transform(X_risco_credito[:, 3])

In [ ]:
X_risco_credito

In [ ]:
import pickle
with open('risco_credito.pkl', 'wb') as f:
    pickle.dump([X_risco_credito, Y_risco_credito], f)

In [ ]:
naive_risco_credito = GaussianNB()
naive_risco_credito.fit(X_risco_credito, Y_risco_credito)                                   

In [ ]:
previsao = naive_risco_credito.predict([[0, 0, 1, 2], [2, 0, 0, 0]])
previsao

In [ ]:
naive_risco_credito.classes_

In [ ]:
naive_risco_credito.class_count_

In [ ]:
naive_risco_credito.class_prior_

### Base credit data

In [ ]:
import pickle
with open('credit.pkl', 'rb') as f:
    X_credit_treinamento, Y_credit_treinamento, X_credit_teste, Y_credit_teste = pickle.load(f)

In [ ]:
X_credit_treinamento.shape, Y_credit_treinamento.shape

In [ ]:
X_credit_teste.shape, Y_credit_teste.shape

In [ ]:
naive_credit_data = GaussianNB()
naive_credit_data.fit(X_credit_treinamento, Y_credit_treinamento)

- Nos comandos acima, nós instânciamos o algoritmo de Naive Bayes com o método de Gauss.
- O comando `fit` faz o treinamento do algoritmo usando os parâmetros escolhidos e gera uma ``tabela de probabilidade``

- Com o algoritmo treinado, podemos usar o comando `predict` para fazer previsões de dados na nova base de dados
- Lembrando que os dados que serão usados para as previsões são os dados de ``teste`` e não os dados de `treinamento` 

In [ ]:
previsoes = naive_credit_data.predict(X_credit_teste)
previsoes

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
accuracy_score(Y_credit_teste, previsoes)

In [ ]:
confusion_matrix(Y_credit_teste, previsoes)

In [ ]:
print(classification_report(Y_credit_teste, previsoes))

### Base census

In [ ]:
with open('census.pkl', 'rb') as f:
    X_census_treinamento, Y_census_treinamento, X_census_teste, Y_census_teste = pickle.load(f)

In [ ]:
X_census_treinamento.shape, Y_census_treinamento.shape

In [ ]:
naive_census_data = GaussianNB()
naive_census_data.fit(X_census_treinamento, Y_census_treinamento)

In [ ]:
census_prev = naive_census_data.predict(X_census_teste)
census_prev

In [ ]:
accuracy_score(Y_census_teste, census_prev)

In [ ]:
confusion_matrix(Y_census_teste, census_prev)

In [ ]:
print(classification_report(Y_census_teste, census_prev))

In [ ]:
from yellowbrick.classifier import ConfusionMatrix

In [ ]:
cm = ConfusionMatrix(naive_census_data)
cm.fit(X_census_treinamento, Y_census_treinamento)
cm.score(X_census_teste, Y_census_teste)

## Aprendizagem por Árvores de Decisão

Para gerarmos a árvore de decisão temos duas fórmulas que podem ser aplicadas: 
- Entropy (entropia)

    $Entropy(S) = \displaystyle\sum_{i=1}^{c}$ - $pi \cdot \log_2pi $

- Gain (ganho de informação)

Após gerarmos uma árvore de decisão, podemos aplicar modificações que chamamos de ``poda (podar uma ávore)`
- **Bias (viés)**

    * Erros por classificação errada

- **Variância**

    * Erros por sensibilidade pequena à mudanças na base de dados de treinamneto

    * Pode levar a ``Overfitting`` (quando o algoritmo se adapta demais a base de treinamento e quando vai para a base de teste comete muitos erros)

- **Vantagens**
    
    * Fácil interpretação

    * Não precisa de normalização ou padronização

    * Rápido para classificar novo registros

- **Desvantagens**

    * Geração de árvores muito complexas

    * Pequenas mudanças nos dados podem mudar a árvore (podar pode ajudar)

    * Problema ``NP-Completo`` para construir a árvore

- Era um método muito popular em meados dos anos 90

- Upgrades como ``random forest (florestas randômicas)`` melhoram o desempenho (usado no Kinect da microsoft)

- CART - Classification and Regression Tress (Ávores para Classficação e Regressão)

### Árvores de decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

#### Base risco de crédito

In [ ]:
import pickle

with open('risco_credito.pkl', 'rb') as f:
    X_risco_credito, Y_risco_credito = pickle.load(f)

In [ ]:
arvore_risco_credito = DecisionTreeClassifier(criterion='entropy')
arvore_risco_credito.fit(X_risco_credito, Y_risco_credito)

In [ ]:
arvore_risco_credito.feature_importances_

In [ ]:
from sklearn import tree
tree.plot_tree(arvore_risco_credito)

In [ ]:
from sklearn import tree
previsores = ['história', 'dívida', 'garantias', 'renda']
tree.plot_tree(arvore_risco_credito, feature_names=previsores)

In [ ]:
from sklearn import tree
previsores = ['história', 'dívida', 'garantias', 'renda']
figura, eixos = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
tree.plot_tree(arvore_risco_credito, feature_names=previsores)

In [ ]:
arvore_risco_credito.classes_

In [ ]:
from sklearn import tree
previsores = ['história', 'dívida', 'garantias', 'renda']
figura, eixos = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
tree.plot_tree(arvore_risco_credito, feature_names=previsores, class_names = arvore_risco_credito.classes_)

In [ ]:
from sklearn import tree
previsores = ['história', 'dívida', 'garantias', 'renda']
figura, eixos = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
tree.plot_tree(arvore_risco_credito, feature_names=previsores, class_names = arvore_risco_credito.classes_, filled=True)

In [ ]:
from sklearn import tree
previsores = ['história', 'dívida', 'garantias', 'renda']
figura, eixos = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
tree.plot_tree(arvore_risco_credito, feature_names=previsores, class_names = arvore_risco_credito.classes_, filled=True); # o ; remove os textos redundantes

In [ ]:
# história boa, dívida alta, garantias nenhuma, renda > 35
# história ruim, dívida alta, garantias adequada, renda < 15

previsoes = arvore_risco_credito.predict([[0, 0, 1, 2], [2, 0, 0, 0]])
previsoes

#### Base credit data

In [ ]:
import pickle

with open('credit.pkl', 'rb') as f:
    X_credit_treinamento, Y_credit_treinamento, X_credit_teste, Y_credit_teste = pickle.load(f)

In [ ]:
arvore_credit = DecisionTreeClassifier(criterion='entropy', random_state=0)

In [ ]:
arvore_credit.fit(X_credit_treinamento, Y_credit_treinamento)

In [ ]:
previsoes = arvore_credit.predict(X_credit_teste)
previsoes

In [ ]:
Y_credit_teste

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(Y_credit_teste, previsoes)
accuracy

In [ ]:
from yellowbrick.classifier import ConfusionMatrix
cm = ConfusionMatrix(arvore_credit)
cm.fit(X_credit_treinamento, Y_credit_treinamento)
cm.score(X_credit_teste, Y_credit_teste)

In [ ]:
print(classification_report(Y_credit_teste, previsoes))

In [ ]:
arvore_credit.classes_

In [ ]:
str(arvore_credit.classes_)

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

previsores = ['income', 'age', 'loan']

# Converta as classes para uma lista
class_names_list = [str(classe) for classe in arvore_credit.classes_]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 20))
tree.plot_tree(arvore_credit, feature_names=previsores, class_names=class_names_list, filled=True)

fig.savefig('arvore_credit.png')


In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

previsores = ['income', 'age', 'loan']

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 20))
tree.plot_tree(arvore_credit, feature_names=previsores, class_names=['0', '1'], filled=True)

fig.savefig('arvore_credit2.png')

#### Base census